In [28]:
import numpy as np
import numpy.random as rnd
import time



def extend_matrix_by_one(M):
    """
    This function extendes the matrix M by adding new row and new column to the end.
    Column and row consist of zeros except the one on the diagonal.
    """

    M = np.hstack((M, np.zeros((M.shape[0],1))))
    M = np.vstack((M, np.array((0.0,)*(M.shape[1]-1) + (1.0,), ndmin=2 )))

    return M

class SVD_updater(object):

    def __init__(self, U,S,Vh, update_V = False, reorth_step=100):

        self.outer_U = U
        self.outer_Vh = Vh

        self.inner_U = None
        self.inner_Vh = None
        self.inner_V_new_pseudo_inverse = None # pseudo inverse of the inner_V

        self.S = S
        self.m = U.shape[0] # number of rows. This quantity doe not change
        self.n = Vh.shape[1] # number of columns. This quantity changes every time we add column

        self.update_Vh = update_V
        #assert update_V == False, "Updarting V is corrently not supported"

        self.reorth_step = reorth_step
        self.update_count = 0 # how many update columns have been called.
        self.reorth_count = 0 # how many update columns since last reorthogonalizetion have been called
                              #This counter is needed to decide when perform an orthogonalization

    def add_column(self, new_col):
        
        self.zero_epsilon = np.sqrt( np.finfo(np.float64).eps * self.m**2 * self.n * 10*2) # epsilon to determine when rank not increases

        old_shape = new_col.shape
        new_col.shape = (old_shape[0],)

        m_vec = np.dot(self.outer_U.T, new_col) # m vector in the motes
        if not self.inner_U is None:
            m_vec = np.dot( self.inner_U.T, m_vec )

        if self.n < self.m: # new column
            if not self.inner_U is None:
                new_u = new_col -  np.dot( self.outer_U, np.dot(  self.inner_U, m_vec ) )
            else:
                new_u = new_col - np.dot( self.outer_U, m_vec )

            mu = np.linalg.norm( new_u, ord=2 )

            # rank is not increased.
            if (np.abs(mu) < self.zero_epsilon ): # new column is from the same subspace as the old column
                rank_increases = False
                U1, self.S, V1 = _SVD_upd_diag( self.S, m_vec, new_col=False )

            else: # rank is increased

                rank_increases = True
                S = np.concatenate( (self.S, (0.0,) ) )
                m_vec = np.concatenate((m_vec, (mu,) ))


                U1, self.S, V1 = _SVD_upd_diag( S, m_vec, new_col=True)

                
                # Update outer matrix
                self.outer_U = np.hstack( (self.outer_U, new_u[:,np.newaxis] / mu) ) # update outer matrix in case of rank increase

            if self.inner_U is None:
                self.inner_U = U1
            else:
                if rank_increases:
                    self.inner_U = extend_matrix_by_one(self.inner_U)

                self.inner_U = np.dot( self.inner_U, U1)

            if self.update_Vh:
                if rank_increases:
                    self.outer_Vh = extend_matrix_by_one(self.outer_Vh)
                    if not self.inner_Vh is None:
                        self.inner_Vh = np.dot( V1.T, extend_matrix_by_one(self.inner_Vh) )
                    else:
                        self.inner_Vh = V1.T

                    if not self.inner_V_new_pseudo_inverse is None:
                        self.inner_V_new_pseudo_inverse = np.dot( V1.T, extend_matrix_by_one(self.inner_V_new_pseudo_inverse) )
                    else:
                        self.inner_V_new_pseudo_inverse = V1.T
                else:
                    r = V1.shape[1]
                    W = V1[0:r,:]
                    w = V1[r,:]; w.shape= (1, w.shape[0]) # row

                    w_norm2 = np.linalg.norm(w, ord=2)**2
                    W_pseudo_inverse = W.T + np.dot( (1/(1-w_norm2))*w.T, np.dot(w,W.T) )
                    del r, w_norm2

                    if not self.inner_Vh is None:
                        self.inner_Vh = np.dot( W.T, self.inner_Vh)
                    else:
                        self.inner_Vh = W.T

                    if not self.inner_V_new_pseudo_inverse is None:
                        self.inner_V_new_pseudo_inverse = np.dot(W_pseudo_inverse, self.inner_V_new_pseudo_inverse)
                    else:
                        self.inner_V_new_pseudo_inverse = W_pseudo_inverse

                    self.outer_Vh = np.hstack( (self.outer_Vh, np.dot( self.inner_V_new_pseudo_inverse.T, w.T)) )

                    del w, W_pseudo_inverse, W

        else: #  self.n > self.m

            U1, self.S, V1 = _SVD_upd_diag( self.S, m_vec, new_col=False)

            if self.inner_U is None:
                self.inner_U = U1
            else:
                self.inner_U = np.dot( self.inner_U, U1)

            if self.update_Vh:
                r = V1.shape[1]
                W = V1[0:r,:]
                w = V1[r,:]; w.shape= (1, w.shape[0]) # row

                w_norm2 = np.linalg.norm(w, ord=2)**2
                W_pseudo_inverse = W.T + np.dot( (1/(1-w_norm2))*w.T, np.dot(w,W.T) )
                del r, w_norm2

                if not self.inner_Vh is None:
                    self.inner_Vh = np.dot( W.T, self.inner_Vh )
                else:
                    self.inner_Vh = W.T

                if not self.inner_V_new_pseudo_inverse is None:
                    self.inner_V_new_pseudo_inverse = np.dot(W_pseudo_inverse, self.inner_V_new_pseudo_inverse)
                else:
                    self.inner_V_new_pseudo_inverse = W_pseudo_inverse

                self.outer_Vh = np.hstack( (self.outer_Vh, np.dot( self.inner_V_new_pseudo_inverse.T, w.T) ) )

                del w, W_pseudo_inverse, W

        self.n = self.outer_Vh.shape[1]
        new_col.shape  = old_shape

        self.update_count += 1 
        self.reorth_count += 1

        if self.reorth_count >= self.reorth_step:
            self._reorthogonalize()
            self.reorth_count = 0
        

    def get_current_svd(self):
        """
            U, S, Vh - SVD. Vh is none if it is not updated.
        """

        if not self.inner_U is None:
            Ur = np.dot( self.outer_U, self.inner_U)
        else:
            Ur = self.outer_U

        if self.update_Vh:
            if not self.inner_Vh is None:
                Vhr = np.dot( self.inner_Vh, self.outer_Vh)
            else:
                Vhr = self.outer_Vh
        else:
            Vhr = None


        return Ur, self.S, Vhr


    def _reorthogonalize(self):

        if not self.inner_U is None:
            US = np.dot( self.inner_U, np.diag(self.S) )
            (Utmp,Stmp,Vhtmp) = np.linalg.svd(US, full_matrices=False, compute_uv=True)

            self.inner_U = Utmp
            self.S = Stmp

            # update outer matrix ->
            self.outer_U = np.dot( self.outer_U, self.inner_U)
            self.inner_U = None
            # update outer matrix <-

            if self.update_Vh:
                self.inner_Vh = np.dot( Vhtmp, self.inner_Vh)

                # update outer matrix ->
                self.outer_Vh = np.dot( self.inner_Vh, self.outer_Vh)
                self.inner_Vh = None
                self.inner_V_new_pseudo_inverse = None
                # update outer matrix <-

            if self.update_Vh:
                return self.outer_U, self.S, self.outer_Vh
            else:
                return self.outer_U, self.S, None
        else:
            pass

    def reorth_was_pereviously(self):
        """
        Function for external testing.

        Functions returns the boolean value which tells
        whether reorthogonalization has performed on the previous step.
        """

        return (self.reorth_count == 0)

func_calls = []
it_count = []



def _SVD_upd_diag(sigmas, m_vec, new_col=True ):

    if new_col == False:
        n_vec = np.copy( m_vec )
        n_vec = n_vec.reshape( n_vec.shape[ 0 ], 1 )

        Si = np.hstack( (np.diag( sigmas ), n_vec  ) )

        Ux, sx, Vtx = np.linalg.svd( Si, full_matrices=False )
        return Ux, sx, Vtx.T

    else:

        Se_i = np.diag( sigmas )
        Se_i[ :, -1 ] = m_vec

        Ux, sx, Vtx = np.linalg.svd( Se_i, full_matrices=False )
        return Ux, sx, Vtx.T


def test_SVD_updater():
    """
    Testinf function for SVD_updater class.
    """

    increaseRank = False

    if increaseRank:
    
        # Test column update. Thin matrix, rank increases
        n_rows = 1000; n_cols = 800
        A = rnd.rand(n_rows,n_cols)
        (U,S,Vh) = np.linalg.svd( A , full_matrices=False, compute_uv=True, overwrite_a=False, check_finite=False )
        
        a1 = rnd.rand(n_rows,1)
        A = np.hstack( (A,a1) )
        (Ut1,St1,Vht1) = np.linalg.svd( A  , full_matrices=False, compute_uv=True, overwrite_a=False, check_finite=False )

        print ( "START TEST 1 -------------------------------" );
        SVD_upd = SVD_updater( U,S,Vh, update_V = True, reorth_step=10)        

        SVD_upd.add_column(a1 )
        (Us1, Ss1, Vhs1) = SVD_upd.get_current_svd()

        
        

        
        Ar1 = np.dot( Us1, np.dot(np.diag(Ss1), Vhs1 ) )
        print ( "END TEST 1 -------------------------------" );        
        diff1 = np.max( np.abs( A - Ar1) )/St1[0]
        print( np.allclose( A, Ar1 ) )
        
        a2 = rnd.rand(n_rows,1)
        A = np.hstack( (A,a2) )
        
        print ( "START TEST 2 -------------------------------" );
        SVD_upd.add_column( a2 )
        (Us2, Ss2, Vhs2) = SVD_upd.get_current_svd()
        
        Ar2 = np.dot( Us2, np.dot(np.diag(Ss2), Vhs2 ) )
        print ( "END TEST 2 -------------------------------" );
        diff2 = np.max( np.abs( A - Ar2) )/St1[0]
        print( np.allclose( A, Ar2 ) )
        
    else:

        #Test column update. Thin matrix, rank not increases
        n_rows = 1000; n_cols = 800
        A = rnd.rand(n_rows,n_cols)
        (U,S,Vh) = np.linalg.svd( A , full_matrices=False, compute_uv=True, overwrite_a=False, check_finite=False )
        
        ##a1 = rnd.rand(5,1)
        a1 = np.dot(A, rnd.rand(n_cols,1) )
        A = np.hstack( (A,a1) )
        (Ut,St,Vht) = np.linalg.svd( A  , full_matrices=False, compute_uv=True, overwrite_a=False, check_finite=False )
        

        print ( "START TEST 1 -------------------------------" );
        SVD_upd = SVD_updater( U,S,Vh, update_V = True, reorth_step=10)
        SVD_upd.add_column( a1 )
        ( Us1, Ss1, Vhs1) = SVD_upd.get_current_svd()

        Ar1 = np.dot( Us1, np.dot(np.diag(Ss1), Vhs1 ) )
        
        diff1 = np.max( np.abs( A - Ar1) )/St[0]
        print ( "END TEST 1 -------------------------------" );
        print( np.allclose( A, Ar1 ) )

        
        
        print ( "START TEST 2 -------------------------------" );
        a2 = rnd.rand(n_rows,1)
        
        #a2 = np.dot(A, np.array([2,1,4,-3],ndmin=2 ).T )
        A = np.hstack( (A,a2) )
        (Ut,St,Vht) = np.linalg.svd( A  , full_matrices=False, compute_uv=True, overwrite_a=False, check_finite=False )
        SVD_upd.add_column( a2 )
        (Us2, Ss2, Vhs2) = SVD_upd.get_current_svd()

        
        Ar2 = np.dot( Us2, np.dot(np.diag(Ss2), Vhs2 ) )
        print ( "END TEST 2 -------------------------------" );        
        print( np.allclose( A, Ar2 ) )
        
        diff2 = np.max( np.abs( A - Ar2) )/St[0]

    print( diff2 )
    return diff2

In [29]:
p = 100; q = 1;
r = min( p, q )

# construct a random matrix of size m, n
#X = np.random.randn( p, q ) + 1j*np.random.randn( p, q )
X = np.random.randn( p, q )

U, S, Vh = np.linalg.svd( X, full_matrices=False )

isvd = SVD_updater( U, S, Vh, update_V = True, reorth_step=100  )

# construct a random update vector
#a = np.random.randn( p, 1 ) + 1j*np.random.randn( p, 1 )
a = np.random.randn( p, 1 )

isvd.add_column( a )
(Un, Sn, Vhn ) = isvd.get_current_svd()

In [30]:
b = np.zeros( (q+1,1) ); b[ q ] = 1
Aab = np.append( X, np.zeros( (p,1), dtype=complex), axis=1  ) + np.dot( a, np.transpose(b) )

X = Aab

Sp = np.diag( isvd.S )
Ai = np.dot( Un, np.dot(np.diag(Sn), Vhn ) )

Aix = np.dot( isvd.outer_U, isvd.inner_U)
Aix = np.dot( Aix, np.diag(isvd.S) )
Aix = np.dot( Aix, np.dot(isvd.inner_Vh, isvd.outer_Vh) )

print( Aab.shape,Ai.shape )
print( np.allclose( Aab, Ai ) )
print( np.allclose( Aab, Aix ) )

(100, 2) (100, 2)
True
True


In [31]:
import time

t0a = time.time()

for i in range(0, 1000):
    # construct a random update vector
    #a = np.random.randn( p, 1 ) + 1j*np.random.randn( p, 1 )
    a = np.random.randn( p, 1 )

    
    t0i = time.time()
    isvd.add_column( a )
    
    (Un, Sn, Vhn ) = (None, None, None)
    if (isvd.inner_U is None) or (isvd.inner_Vh is None):
        (Un, Sn, Vhn ) = isvd.get_current_svd()
    t1i = time.time()

    
    q = q+1

    b = np.zeros( (q+1,1) ); b[ q ] = 1
    Aab = np.append( Aab, np.zeros( (p,1), dtype=complex), axis=1  ) + np.dot( a, np.transpose(b) )

    
    # t0 = time.time()
    # Uab, sab, Vtab = np.linalg.svd( Aab, full_matrices=False )
    # t1 = time.time()
    
    
    # print( "Current index: ", i)
    # print( "Incremental: ", (t1i-t0i), " Regular: ", (t1-t0))
    # print( "Shapes: ", Aab.shape,Ai.shape )

    if not Un is None:
        Sp = np.diag( isvd.S )
        Ai = np.dot( Un, np.dot(np.diag(Sn), Vhn ) )
        # print( "Aab vs Ai: ", np.allclose( Aab, Ai ) )

    if Un is None:
        Aix = np.dot( isvd.outer_U, isvd.inner_U)
        Aix = np.dot( Aix, np.diag(isvd.S) )
        Aix = np.dot( Aix, np.dot(isvd.inner_Vh, isvd.outer_Vh) )

        # print( "Aab vs Aix: ", np.allclose( Aab, Aix ) )
    # print( "---------------------------------\n" )

t1a = time.time()
print( "Overall time: ", (t1a-t0a) )

Overall time:  3.588779926300049
